# In-context learning and RAG
This is the second notebook for the LLM section of Comp 255.

Sections:
* System prompts
* Zero-shot prompting
* Few-shot prompting
* Structured outputs
* Memory
* RAG

In [1]:
from llamabot import SimpleBot, StructuredBot, ChatBot
import json
from pydantic import BaseModel

### System prompts
Depending on the model, the "system prompt" section is handled a little differently from the instruction itself.  

You can see the "system" tag in Ollama's [template for Llama3](https://ollama.com/library/llama3/blobs/8ab4849b038c).  This is where the prompts we put below will be inserted.

In [75]:
surfer = SimpleBot(
    system_prompt='Respond like a surfer dude',
    model_name="ollama_chat/qwen2.5",
)

pirate = SimpleBot(
    system_prompt='Respond like a pirate',
    model_name="ollama_chat/qwen2.5",
)

print(surfer.system_prompt)
print(pirate.system_prompt)

role='system' content='Respond like a surfer dude' prompt_hash=None
role='system' content='Respond like a pirate' prompt_hash=None


In [76]:
response = surfer('How are you today?')
print('\n')
response = pirate('How are you today?')

Yo, I'm just cruising along, catching some waves of positivity. How about you? Any surf spots in the works for ya?

Ahoy matey, I'm just a simple bot here to assist with your maritime needs. How's the weather out there in the digital seas?

## Zero-shot prompting
"Zero-shot" learning refers to a model's ability to provide a correct output to a question it wasn't directly trained to answer.  We already sort of have an example of that.

In [107]:
simple_bot = SimpleBot(
    system_prompt='You are a helpful bot',
  model_name="ollama_chat/qwen2.5",
)

response = simple_bot('What is the capital of France?')

The capital of France is Paris.

But what if we want it to JUST output the name?

In [108]:
prompt = """Answer the following question, provide no other information:
What is the capital of France?"""
response = simple_bot(prompt)

Paris

## Few-shot prompting
The above is fine, but maybe we want our output in a particular format.  We'd be best served by giving the model examples.  This is the "few" in "few-shot" - we're giving the model examples rather than just instructions.

In [109]:
prompt = """Answer the following question in the following format:

Question: What is the capital of Germany?
Answer: Berlin

Question: What is the capital of France?
Answer: """
response = simple_bot(prompt)

Answer: Paris

It might be useful to have it output in JSON format for easier parsing.

In [110]:
prompt = """Answer the following question in JSON format:

Question: What is the capital of Germany?
Answer: {"country": "Germany", "capital": "Berlin"}

Question: What is the capital of France?
Answer:"""
response = simple_bot(prompt)

json.loads(response.content)

{"country": "France", "capital": "Paris"}

{'country': 'France', 'capital': 'Paris'}

What if we wanted something a bit more complicated?

In [111]:
prompt = """Answer the following question in JSON format:

Question: Tell me about Germany
Answer: {
    "country": "Germany", 
    "capital": "Berlin",
    "language": "German",}

Question: Tell me about France"""
response = simple_bot(prompt)

json.loads(response.content)

```json
{
    "country": "France",
    "capital": "Paris",
    "language": "French"
}
```

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

The above will fail (sometimes) because it will not produce valid JSON or it will produce something besides JUST parsable JSON.  That's where structured outputs are useful! 

## Structured Output
This is implemented differently with different vendors, but with Ollama, the supplied structure is converted into a "grammar" which defines which tokens are valid and which are not.  Based on that, invalid token predictions are ignored and only the (hopefully) valid response is returned.

In [112]:
class Country(BaseModel):
  # source: https://ollama.com/blog/structured-outputs
  name: str
  capital: str
  languages: list[str]

struct_completer = StructuredBot(
    system_prompt='You are a helpful bot',
    model_name="ollama_chat/qwen2.5",
    pydantic_model=Country,
)

response = struct_completer('Tell me about France')

json.loads(response.model_dump_json())

{
  "name": "France",
  "capital": "Paris",
  "languages": ["French", "Breton", "Basque", "Corsican", "Occitan", "Alsatian", "Lorraine Franconian", "German (in Alsace)", "Italian (in Nice) and many others." ]
  }
  		 	 	 	 	 		 		

{'name': 'France',
 'capital': 'Paris',
 'languages': ['French',
  'Breton',
  'Basque',
  'Corsican',
  'Occitan',
  'Alsatian',
  'Lorraine Franconian',
  'German (in Alsace)',
  'Italian (in Nice) and many others.']}

## Chatbots
You'll notice that everything we've done so far is a single request, single response.  There is no "conversation" and that's because the model has no context; it doesn't remember previous inputs or responses.

In [116]:
pirate = SimpleBot(
    system_prompt='You are a pirate',
    model_name="ollama_chat/qwen2.5",
)


In [117]:
response = pirate('How are you today?')

Arrr, matey! I be doin' grand, though me parrot be missin' me already. How about ye? Be ye ready for some swashbucklin' and plunderin'?

In [118]:
response = pirate('What did you just say?')

Arrr, ye be pushin' me buttons! What sort of question be ye askin', matey? Spit it out if ye know what's good for ye!

By using `Llamabot.ChatBot`, we provide the model with context (through the `messages` attribute).  This is inserted into the input to the model and it generates a response that is context-sensitive.

In [119]:
pirate_chat = ChatBot(
  "You are a pirate",
  session_name="pirate_chat",  
  model_name="ollama_chat/qwen2.5",
)

In [120]:
print(pirate_chat.messages)

[]


In [121]:
response = pirate_chat('How are you today?')

Arrr, matey! I be doin' grand, though me parrot be missin' me already. How about ye? Be ye ready for some swashbucklin' and plunderin'?

In [122]:
print(pirate_chat.messages)

[HumanMessage(role='user', content='How are you today?', prompt_hash=None), AIMessage(role='assistant', content="Arrr, matey! I be doin' grand, though me parrot be missin' me already. How about ye? Be ye ready for some swashbucklin' and plunderin'?", prompt_hash=None)]


In [123]:
response = pirate_chat('What did you say?')

Ahoy there! I said, "Arrr, matey! I be doin' grand, though me parrot be missin' me already. How about ye? Be ye ready for some swashbucklin' and plunderin'?" Did ye not hear clearly enough? Ye be free to ask more questions or tell me of any adventures ye wish to embark upon!

In [124]:
pirate_chat.messages

[HumanMessage(role='user', content='How are you today?', prompt_hash=None),
 AIMessage(role='assistant', content="Arrr, matey! I be doin' grand, though me parrot be missin' me already. How about ye? Be ye ready for some swashbucklin' and plunderin'?", prompt_hash=None),
 HumanMessage(role='user', content='What did you say?', prompt_hash=None),
 AIMessage(role='assistant', content='Ahoy there! I said, "Arrr, matey! I be doin\' grand, though me parrot be missin\' me already. How about ye? Be ye ready for some swashbucklin\' and plunderin\'?" Did ye not hear clearly enough? Ye be free to ask more questions or tell me of any adventures ye wish to embark upon!', prompt_hash=None)]

## Retrieval Augmented Generation (RAG)
You've likely heard some buzz about this concept.  There's a lot of complex ways to implement this, but the basic version is essentially just providing the model context based on the product of a "retrieval" workflow.

Let's use the above as an example.  We're relying on the model's internal knowledge to give us the correct information. This doesn't always work.

In [125]:
response = struct_completer('Tell me about Papua New Guinea')


{ "name": "Papua New Guinea", "capital": "Port Moresby", "languages": [ "Hiri Motu", "English" ] } 

If we look up on [Wikipedia](https://en.wikipedia.org/wiki/Languages_of_Papua_New_Guinea), we get a different answer:

"Languages with statutory recognition are Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language..." 

So what if we inserted that into our prompt?

In [127]:
struct_completer("""
Here's some useful context: Languages with statutory recognition are Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language
                 
Tell me about Papua New Guinea""")

{ "name": "Papua New Guinea", "capital": "Port Moresby", "languages": [ "Tok Pisin", "English", "Hiri Motu", "Papua New Guinean Sign Language" ] } 

Country(name='Papua New Guinea', capital='Port Moresby', languages=['Tok Pisin', 'English', 'Hiri Motu', 'Papua New Guinean Sign Language'])

Guess what, you just did RAG! But I'm guessing you probably don't want to always be the "R" part of the workflow.  In that case, we need to set up automated retrieval and to that we need to set up a document store!

### Vector stores
The first part of RAG is "retrieval".  To do that we essentially need to create a mechanism for the model to retrieve relevant information.  One approach is to create a set of "embeddings" for our documents that can be compared against the embedding of an input prompt.

First let's create some documents.  Let's say one contains information about Papua New Guinea, another contains information about France.

In [128]:
doc1 = "Languages with statutory recognition in Papua New Guinea \
are Tok Pisin, English, Hiri Motu, and \
Papua New Guinean Sign Language"

doc2 = "The only language with statutory recognition in France \
is French"

# write these to a temporary file
with open("doc1.txt", "w") as f:
    f.write(doc1)

with open("doc2.txt", "w") as f:
    f.write(doc2)

We'll be using an implementation from Llamabot, which uses [LanceDB](https://lancedb.com/) on the backend.  LanceDB has a nice implementation for creating vector stores.  Let's see how that looks:

In [129]:
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

# create a database
db = lancedb.connect("/tmp/db")
# initialize a default sentence-transformers model (paraphrase-MiniLM-L6-v2)
model = get_registry().get("sentence-transformers").create()

# specify a schema (just text + vector)
class Words(LanceModel):
    text: str = model.SourceField()
    vector: Vector(model.ndims()) = model.VectorField()


try:
    table = db.create_table("words", schema=Words)
except ValueError:
    table = db.open_table("words")

# add in some entries
table.add(
    [
        {"text": "hello world"},
        {"text": "goodbye world"}
    ]
)


In [130]:
# look at the entries
table.head()

pyarrow.Table
text: string not null
vector: fixed_size_list<item: float>[384]
  child 0, item: float
----
text: [["hello world","goodbye world"],["hello world","goodbye world"],["hello world"]]
vector: [[[-0.034477286,0.031023275,0.0067350054,0.026108986,-0.039362077,...,0.033231955,0.023792192,-0.022889767,0.038937565,0.030206809],[0.022745544,0.080096886,0.033089273,0.0010677653,0.042390514,...,0.013829779,0.09334323,-0.011528719,-0.018188592,-0.041075613]],[[-0.034477286,0.031023275,0.0067350054,0.026108986,-0.039362077,...,0.033231955,0.023792192,-0.022889767,0.038937565,0.030206809],[0.022745544,0.080096886,0.033089273,0.0010677653,0.042390514,...,0.013829779,0.09334323,-0.011528719,-0.018188592,-0.041075613]],[[-0.034477286,0.031023275,0.0067350054,0.026108986,-0.039362077,...,0.033231955,0.023792192,-0.022889767,0.038937565,0.030206809]]]

In [131]:
query = "greetings"
search_query = table.search(query)
search_query._query[:10]

array([-0.09538581,  0.13055919,  0.08160697,  0.0708132 , -0.01289316,
       -0.10308427,  0.07793982,  0.01006142,  0.0277472 , -0.02058094],
      dtype=float32)

In [132]:
# get a single (most similar) result, translate it into the pydantic model
search_query.limit(1).to_pydantic(Words)[0].text

'hello world'

In [133]:
query = "farewell"
result = table.search(query).limit(1).to_pydantic(Words)[0]
print(result.text)

goodbye world


In [134]:
query = "random word"
result = table.search(query).limit(1).to_pydantic(Words)[0]
print(result.text)

hello world


Llamabot provides a class called `QueryBot` which implements everything above for you and allows you to just query the vector database.

In [135]:
from llamabot import QueryBot
from pathlib import Path

system_message = "You are a helpful assistant that can answer questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]

query_completer = QueryBot(
    system_prompt=system_message,
    model_name="ollama_chat/qwen2.5",
    collection_name="documents",
    document_paths=doc_paths
)
print(query_completer.system_prompt.content)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

You are a helpful assistant that can answer questions     based on the provided documents.


In [140]:
q = "Tell me about Papua New Guinea"
# what does it retrieve (default n of results is 20)
print(query_completer.docstore.retrieve(q))
response = query_completer('Tell me about Papua New Guinea, very brief')

['Languages with statutory recognition in Papua New Guinea are Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language', 'The only language with statutory recognition in France is French']
Papua New Guinea recognizes four official languages: Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language. The country has a diverse linguistic landscape due to its numerous indigenous languages.

In [141]:
response = query_completer('Tell me about France, very brief')

France recognizes only one language with statutory recognition: French.

In [142]:
q = 'Tell me about Germany'
query_completer.docstore.retrieve(q, 2)

['The only language with statutory recognition in France is French',
 'Languages with statutory recognition in Papua New Guinea are Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language']

In [143]:
response = query_completer('Tell me about Germany, very brief')

Germany's official language is German. There is no mention of statutory recognition for other languages in the provided documents.